# ESP32/Arduino API Controller

This notebook controls ESP32/Arduino controllers via HTTP requests. It supports servos, stepper motors, LEDs, and button status queries, based on the provided API.

## WiFi Configuration

Here you can configure the WiFi settings for your ESP32/Arduino.

In [1]:
import requests
import time
import json

# WiFi Configuration
WIFI_SSID = "MyWiFi"  # Enter your WiFi SSID here
WIFI_PASSWORD = "MyPassword"  # Enter your WiFi password here

# Base URL for all API requests - Adjust the IP address to your ESP32/Arduino
BASE_URL = "http://192.168.137.232"

def configure_wifi(ssid=None, password=None):
    """
    Configures the WiFi connection of the ESP32/Arduino.
    
    Args:
        ssid (str): WiFi SSID/network name
        password (str): WiFi password
        
    Returns:
        dict or str: The response from the controller or None on error
    """
    if ssid is None:
        ssid = WIFI_SSID
    if password is None:
        password = WIFI_PASSWORD
        
    params = {"ssid": ssid, "password": password}
    
    try:
        url = f"{BASE_URL}/configureWifi"
        response = requests.post(url, json=params, timeout=5)
        response.raise_for_status()  # Raises exception for HTTP errors
        
        print(f"WiFi configuration sent. SSID: {ssid}")
        
        try:
            return response.json()  # Try to parse JSON
        except ValueError:
            return response.text  # If not a JSON response
    
    except requests.exceptions.RequestException as e:
        print(f"Error in WiFi configuration: {e}")
        return None

In [2]:
# Send WiFi configuration
# Replace the values with your own WiFi data
configure_wifi("MyWiFi", "MyPassword")

Error in WiFi configuration: 404 Client Error: Not Found for url: http://192.168.137.232/configureWifi


## 1. Import Libraries

We only need a few libraries that are available by default in most Python environments.

In [3]:
# Libraries have already been imported
# All required libraries:
# 1. requests - For HTTP requests to the ESP32/Arduino
# 2. time - For delays (time.sleep)
# 3. json - For processing JSON responses

# These libraries were already imported in cell 2 and
# are available in all subsequent cells

## 2. Helper Functions for API Requests

These functions send HTTP requests to the various API endpoints and return the responses.

In [4]:
def make_api_request(endpoint, params=None):
    """
    Sends a GET request to the specified API endpoint.
    
    Args:
        endpoint (str): The API endpoint without leading slash
        params (dict, optional): Parameters for the request
        
    Returns:
        dict or str: The response from the controller or None on error
    """
    url = f"{BASE_URL}/{endpoint}"
    
    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()  # Raises exception for HTTP errors
        
        try:
            return response.json()  # Try to parse JSON
        except ValueError:
            return response.text  # If not a JSON response
    
    except requests.exceptions.RequestException as e:
        print(f"Error in API request: {e}")
        return None

## 3. Control Servo

The servo motor can be positioned at various angles between 0 and 90 degrees.

In [5]:
def set_servo_angle(angle):
    """
    Controls the servo motor to a specific angle.
    
    Args:
        angle (int): Angle between 0 and 90 degrees
        
    Returns:
        The response from the controller
    """
    if not 0 <= angle <= 90:
        print("Error: The angle must be between 0 and 90 degrees.")
        return None
    
    params = {"angle": angle}
    response = make_api_request("setServo", params)
    
    print(f"Servo set to {angle} degrees.")
    print(f"Response: {response}")
    
    return response

# Example: Set servo to 0 degrees
set_servo_angle(0)

Servo set to 0 degrees.
Response: Servo positioned to 0 degrees


'Servo positioned to 0 degrees'

In [6]:
# Set servo to middle position (45 degrees)
set_servo_angle(45)

Servo set to 45 degrees.
Response: Servo positioned to 45 degrees


'Servo positioned to 45 degrees'

## 4. Control Stepper Motor

The stepper motor can be controlled with different step counts, directions, and speeds.

In [7]:
def move_stepper(steps, direction=1, speed=None):
    """
    Controls the stepper motor.
    
    Args:
        steps (int): Number of steps
        direction (int): 1 for up, -1 for down
        speed (int, optional): Speed (if supported)
        
    Returns:
        The response from the controller
    """
    if steps < 0:
        print("Error: The number of steps must be positive.")
        return None
    
    if direction not in [1, -1]:
        print("Error: The direction must be 1 (up) or -1 (down).")
        return None
    
    params = {
        "steps": steps,
        "direction": direction
    }
    
    if speed is not None:
        params["speed"] = speed
    
    response = make_api_request("setMotor", params)
    
    dir_text = "up" if direction == 1 else "down"
    speed_text = f" with speed {speed}" if speed is not None else ""
    print(f"Stepper motor moves {steps} steps {dir_text}{speed_text}.")
    print(f"Response: {response}")
    
    return response

# Example: 4096 steps up (360 degrees)
move_stepper(4096, 1)

Stepper motor moves 4096 steps up.
Response: Motor moved 4096 steps in direction 1 with speed 100%


'Motor moved 4096 steps in direction 1 with speed 100%'

In [8]:
# 4096 steps down with speed
move_stepper(4096, -1, 80)

Error in API request: HTTPConnectionPool(host='192.168.137.232', port=80): Read timed out. (read timeout=5)
Stepper motor moves 4096 steps down with speed 80.
Response: None


## 5. Set LED Color

The RGB LED can be controlled with hexadecimal color values.

In [9]:
def set_led_color(color_hex):
    """
    Sets the color of the RGB LED.
    
    Args:
        color_hex (str): Hexadecimal color code (e.g. "#FF0000" for red)
        
    Returns:
        The response from the controller
    """
    # Make sure the color has a '#'
    if not color_hex.startswith("#"):
        color_hex = "#" + color_hex
    
    params = {"hex": color_hex}
    response = make_api_request("hexcolor", params)
    
    print(f"LED color set to {color_hex}.")
    print(f"Response: {response}")
    
    return response

# Example: Set LED to white
set_led_color("#FFFFFF")

LED color set to #FFFFFF.
Response: Color changed to #FFFFFF


'Color changed to #FFFFFF'

In [10]:
# Set LED to red
set_led_color("#FF0000")

LED color set to #FF0000.
Response: Color changed to #FF0000


'Color changed to #FF0000'

In [11]:
# Set LED to green
set_led_color("#00FF00")

LED color set to #00FF00.
Response: Color changed to #00FF00


'Color changed to #00FF00'

In [12]:
# Set LED to blue
set_led_color("#0000FF")

LED color set to #0000FF.
Response: Color changed to #0000FF


'Color changed to #0000FF'

## 6. Control LED Brightness

The brightness of the LEDs can be adjusted from 0 to 100%.

In [13]:
def set_led_brightness(brightness):
    """
    Sets the brightness of the LED.
    
    Args:
        brightness (int): Brightness level between 0 and 100 percent
        
    Returns:
        The response from the controller
    """
    if not 0 <= brightness <= 100:
        print("Error: Brightness must be between 0 and 100 percent.")
        return None
    
    params = {"value": brightness}
    response = make_api_request("setBrightness", params)
    
    print(f"LED brightness set to {brightness}%.")
    print(f"Response: {response}")
    
    return response

# Example: Set brightness to 25%
set_led_brightness(25)

LED brightness set to 25%.
Response: Brightness set to 25


'Brightness set to 25'

In [14]:
# Set brightness to 100%
set_led_brightness(100)

LED brightness set to 100%.
Response: Brightness set to 100


'Brightness set to 100'

In [15]:
# Set brightness to 50%
set_led_brightness(50)

LED brightness set to 50%.
Response: Brightness set to 50


'Brightness set to 50'

## 7. Query Button/Switch Status

Here the status of the button or switch is queried.

In [16]:
def get_button_state():
    """
    Queries the status of the button.
    
    Returns:
        The status of the button (true/false or 1/0) or the response from the controller
    """
    response = make_api_request("getButtonState")
    
    print(f"Button status: {response}")
    
    return response

# Query button status
get_button_state()

Button status: {'pressed': True}


{'pressed': True}